In [ ]:
# for basic SMILES operations, descriptors and fingerprints
!pip install -q rdkit-pypi

# for graphs
!pip install -q dgl

# for mol2vec
!pip install -q mol2vec

!pip install duckdb

!pip install faiss-gpu

In [ ]:
config = {
    'num_samples_per_class': 10000,  # Adjust as needed
    'ecfp_radius': 3,
    'ecfp_bits': 1024,
    'batch_size': 32,  # Increase batch size if you have more GPU memory
    'train_test_split_ratio': 0.2,
    'molformer_model_name': 'ibm/MoLFormer-XL-both-10pct',
    'use_gpu_for_faiss': True,
    'early_stopping_rounds': 10,  # Add early stopping rounds
    'rf_params': {
        'n_estimators': 250,  # Increase number of trees
        'max_depth': 9,
        'random_state': 42,
        'n_jobs': -1
    },
    'xgb_params': {
        'n_estimators': 250,  # Increase number of boosting rounds
        'max_depth': 6,
        'learning_rate': 0.1,
        'subsample': 0.8,
        'colsample_bytree': 0.8,
        'random_state': 42
    }
}


In [ ]:
import duckdb
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from rdkit import Chem
from rdkit.Chem import AllChem
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

def load_and_prepare_data(config):
    # Load and balance dataset
    train_path = '/kaggle/input/leash-BELKA/train.parquet'
    con = duckdb.connect()

    df = con.query(f"""
        (SELECT *
         FROM parquet_scan('{train_path}')
         WHERE binds = 0
         ORDER BY random()
         LIMIT {config['num_samples_per_class']})
        UNION ALL
        (SELECT *
         FROM parquet_scan('{train_path}')
         WHERE binds = 1
         ORDER BY random()
         LIMIT {config['num_samples_per_class']})
    """).df()

    con.close()

    # Combine SMILES strings
    df['combined_smiles'] = df['molecule_smiles'] + ' ' + df['buildingblock1_smiles'] + ' ' + df['buildingblock2_smiles'] + ' ' + df['buildingblock3_smiles']

    # One-hot encode the protein_name
    onehot_encoder = OneHotEncoder(sparse_output=False)
    protein_onehot = onehot_encoder.fit_transform(df[['protein_name']])
    df['protein_onehot'] = list(protein_onehot)

    # Convert SMILES to RDKit molecules
    df['molecule'] = df['molecule_smiles'].apply(Chem.MolFromSmiles)

    # Function to generate ECFP fingerprints
    def generate_ecfp(molecule, radius=config['ecfp_radius'], bits=config['ecfp_bits']):
        if molecule is None:
            return [0] * bits
        return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))

    # Generate ECFPs
    df['ecfp'] = df['molecule'].apply(lambda x: generate_ecfp(x, config['ecfp_radius'], config['ecfp_bits']))

    # Load pre-trained MolFormer model and tokenizer
    mf_tokenizer = AutoTokenizer.from_pretrained(config['molformer_model_name'], deterministic_eval=True, trust_remote_code=True)
    mf_model = AutoModelForSequenceClassification.from_pretrained(config['molformer_model_name'], num_labels=2, trust_remote_code=True)
    mf_model.eval()

    # Tokenize SMILES and get embeddings
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    mf_model.to(device)
    
    def get_molformer_embeddings(smiles):
        inputs = mf_tokenizer(smiles, padding=True, truncation=True, return_tensors="pt")
        inputs = {key: val.to(device) for key, val in inputs.items()}  # Ensure all inputs are on the same device
        with torch.no_grad():
            outputs = mf_model(**inputs)
        return outputs.logits.cpu().numpy().flatten()

    df['molformer_embeddings'] = df['combined_smiles'].apply(get_molformer_embeddings)

    # Combine ECFPs and MolFormer embeddings
    def combine_features(ecfp, molformer, protein_onehot):
        return np.concatenate([ecfp, molformer, protein_onehot])

    df['hybrid_embeddings'] = df.apply(lambda row: combine_features(row['ecfp'], row['molformer_embeddings'], row['protein_onehot']), axis=1)

    # Create the enhanced_smiles column by concatenating the combined SMILES and the protein_onehot encoding as a string
    protein_str = [' '.join(map(str, row)) for row in df['protein_onehot']]
    df['enhanced_smiles'] = df['combined_smiles'] + ' ' + protein_str

    return df, onehot_encoder

# Load and prepare data
df, onehot_encoder = load_and_prepare_data(config)
df.head()


In [ ]:
import faiss
import numpy as np
import torch
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from sklearn.model_selection import StratifiedKFold, learning_curve, train_test_split
import matplotlib.pyplot as plt
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Function to generate ECFP
def generate_ecfp(molecule, radius, bits):
    if molecule is None:
        return [0] * bits
    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))

# Function to get MolFormer embeddings in batches
def get_molformer_embeddings_batch(smiles_list, tokenizer, model, device):
    inputs = tokenizer(smiles_list, padding=True, truncation=True, max_length=512, return_tensors="pt")
    inputs = {key: val.to(device) for key, val in inputs.items()}  # Move inputs to the same device as the model
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.logits.cpu().numpy()
    return embeddings

# Function to generate features for both training and submission
def generate_features(df, onehot_encoder, config, tokenizer, model, device):
    df['combined_smiles'] = df['molecule_smiles'] + ' ' + df['buildingblock1_smiles'] + ' ' + df['buildingblock2_smiles'] + ' ' + df['buildingblock3_smiles']

    # Generate ECFPs for the molecule_smiles
    df['molecule'] = df['molecule_smiles'].apply(Chem.MolFromSmiles)
    df['ecfp'] = df['molecule'].apply(lambda x: generate_ecfp(x, config['ecfp_radius'], config['ecfp_bits']))

    # One-hot encode the protein_name
    protein_onehot = onehot_encoder.transform(df[['protein_name']])
    protein_onehot_df = pd.DataFrame(protein_onehot, index=df.index)

    # Get MolFormer embeddings for combined SMILES in batches
    combined_smiles_list = df['combined_smiles'].tolist()
    molformer_embeddings_list = []

    for i in range(0, len(combined_smiles_list), config['batch_size']):
        batch_smiles = combined_smiles_list[i:i + config['batch_size']]
        molformer_embeddings_list.extend(get_molformer_embeddings_batch(batch_smiles, tokenizer, model, device))

    df['molformer_embeddings'] = molformer_embeddings_list

    # Combine ECFPs, MolFormer embeddings, and one-hot encoded protein names
    df['hybrid_embeddings'] = df.apply(
        lambda row: np.concatenate([row['ecfp'], row['molformer_embeddings'], protein_onehot_df.loc[row.name]]), axis=1
    )

    return df

# Load and prepare data
def load_and_prepare_data(config):
    import duckdb

    train_path = '/kaggle/input/leash-BELKA/train.parquet'
    con = duckdb.connect()

    df = con.query(f"""
        (SELECT *
         FROM parquet_scan('{train_path}')
         WHERE binds = 0
         ORDER BY random()
         LIMIT {config['num_samples_per_class']})
        UNION ALL
        (SELECT *
         FROM parquet_scan('{train_path}')
         WHERE binds = 1
         ORDER BY random()
         LIMIT {config['num_samples_per_class']})
    """).df()

    con.close()

    # Combine SMILES strings
    df['combined_smiles'] = df['molecule_smiles'] + ' ' + df['buildingblock1_smiles'] + ' ' + df['buildingblock2_smiles'] + ' ' + df['buildingblock3_smiles']

    # One-hot encode the protein_name
    onehot_encoder = OneHotEncoder(sparse_output=False)
    protein_onehot = onehot_encoder.fit_transform(df[['protein_name']])
    df['protein_onehot'] = list(protein_onehot)

    # Convert SMILES to RDKit molecules
    df['molecule'] = df['molecule_smiles'].apply(Chem.MolFromSmiles)

    # Generate ECFPs
    df['ecfp'] = df['molecule'].apply(lambda x: generate_ecfp(x, config['ecfp_radius'], config['ecfp_bits']))

    return df, onehot_encoder

# Load and prepare data
df, onehot_encoder = load_and_prepare_data(config)

# Load pre-trained models and tokenizer
mf_tokenizer = AutoTokenizer.from_pretrained(config['molformer_model_name'], deterministic_eval=True, trust_remote_code=True)
mf_model = AutoModelForSequenceClassification.from_pretrained(config['molformer_model_name'], num_labels=2, trust_remote_code=True)
mf_model.eval()

# Move the model to the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mf_model.to(device)

# Generate features for the training data
df = generate_features(df, onehot_encoder, config, mf_tokenizer, mf_model, device)

# Prepare vector search and RAG features
def prepare_vector_search_and_rag_features(df, config):
    # Convert to list of feature vectors for modeling
    X = df['hybrid_embeddings'].tolist()
    y = df['binds'].tolist() if 'binds' in df.columns else None

    # Convert hybrid embeddings to numpy array and ensure float32 type
    hybrid_embeddings = np.vstack(df['hybrid_embeddings'].values).astype(np.float32)

    # Normalize embeddings
    faiss.normalize_L2(hybrid_embeddings)

    # Initialize FAISS index with GPU
    d = hybrid_embeddings.shape[1]  # Dimension of the embeddings
    res = faiss.StandardGpuResources() if config['use_gpu_for_faiss'] else None
    index = faiss.IndexFlatL2(d)  # Index for flat (exact) L2 distance
    if res:
        gpu_index = faiss.index_cpu_to_gpu(res, 0, index)  # Move index to GPU
    else:
        gpu_index = index

    # Add embeddings to the index
    gpu_index.add(hybrid_embeddings)

    # Retrieve the top k nearest neighbors for each embedding
    k = 5  # Number of nearest neighbors to retrieve
    D, I = gpu_index.search(hybrid_embeddings, k)

    # Apply attention mechanism and weighted averaging
    def apply_attention_and_weighted_averaging(embeddings, neighbors_idx, distances):
        augmented_embeddings = []
        for i, (neighbors, dist) in enumerate(zip(neighbors_idx, distances)):
            neighbor_embeddings = embeddings[neighbors]

            # Calculate attention weights
            similarities = 1 / (dist + 1e-6)
            attention_weights = torch.softmax(torch.tensor(similarities), dim=0).numpy()

            # Weighted averaging of neighbor embeddings
            weighted_average = np.average(neighbor_embeddings, axis=0, weights=attention_weights)

            # Combine the original embedding with the weighted average of neighbors
            final_embedding = np.concatenate([embeddings[i], weighted_average])
            augmented_embeddings.append(final_embedding)

        return np.array(augmented_embeddings)

    final_feature_vectors = apply_attention_and_weighted_averaging(hybrid_embeddings, I, D)

    return final_feature_vectors, np.array(y) if y else None  # Convert y to numpy array if available

# Prepare vector search and RAG features for training
X_final, y = prepare_vector_search_and_rag_features(df, config)

# Train the first ensemble model
def train_first_ensemble_model(X_final, y, config):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    # Store metrics
    all_metrics = []
    
    for train_index, test_index in skf.split(X_final, y):
        X_train, X_test = X_final[train_index], X_final[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Train individual models
        rf_model = RandomForestClassifier(**config['rf_params'])

        # Ensure 'use_label_encoder' and 'eval_metric' are not in config['xgb_params']
        xgb_params = {k: v for k, v in config['xgb_params'].items() if k not in ['use_label_encoder', 'eval_metric', 'early_stopping_rounds']}
        xgb_model = XGBClassifier(tree_method='hist', use_label_encoder=False, eval_metric='logloss', **xgb_params)

        xgb_model.fit(X_train, y_train)
        rf_model.fit(X_train, y_train)

        # Ensemble model
        ensemble_model = VotingClassifier(estimators=[
            ('rf', rf_model), ('xgb', xgb_model)
        ], voting='soft')

        ensemble_model.fit(X_train, y_train)

        # Evaluate model
        metrics = evaluate_model(ensemble_model, X_test, y_test)
        all_metrics.append(metrics)
    
    # Calculate average metrics
    avg_metrics = np.mean(all_metrics, axis=0)
    return ensemble_model, avg_metrics

# Function to evaluate the model
def evaluate_model(model, features, labels):
    predictions = model.predict(features)
    prob_predictions = model.predict_proba(features)[:, 1]
    roc_auc = roc_auc_score(labels, prob_predictions)
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='macro')
    return roc_auc, accuracy, f1

# Train the first ensemble model
ensemble_model, avg_metrics = train_first_ensemble_model(X_final, y, config)
print(f"First Ensemble Average Metrics: ROC AUC: {avg_metrics[0]}, Accuracy: {avg_metrics[1]}, F1 Score: {avg_metrics[2]}")

# Plot learning curve
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None, n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5), ax=None):
    if ax is None:
        ax = plt.gca()

    ax.set_title(title)
    if ylim is not None:
        ax.set_ylim(*ylim)
    ax.set_xlabel("Training examples")
    ax.set_ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes
    )
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    ax.grid()

    ax.fill_between(train_sizes, train_scores_mean - train_scores_std,
                    train_scores_mean + train_scores_std, alpha=0.1,
                    color="r")
    ax.fill_between(train_sizes, test_scores_mean - test_scores_std,
                    test_scores_mean + test_scores_std, alpha=0.1, color="g")
    ax.plot(train_sizes, train_scores_mean, 'o-', color="r",
            label="Training score")
    ax.plot(train_sizes, test_scores_mean, 'o-', color="g",
            label="Cross-validation score")

    ax.legend(loc="best")
    return ax

# Plot learning curve for the ensemble model
cv = StratifiedKFold(n_splits=3)
fig, ax = plt.subplots(figsize=(10, 6))
plot_learning_curve(ensemble_model, "First Ensemble Model Learning Curve", X_final, y, cv=cv, ax=ax)
plt.show()

# Function to prepare test data
def prepare_test_data(df_test, onehot_encoder, config, tokenizer, model, device, expected_dim):
    # Generate ECFPs for the molecule_smiles
    df_test['molecule'] = df_test['molecule_smiles'].apply(Chem.MolFromSmiles)
    df_test['ecfp'] = df_test['molecule'].apply(lambda x: generate_ecfp(x, config['ecfp_radius'], config['ecfp_bits']))

    # One-hot encode the protein_name
    protein_onehot = onehot_encoder.transform(df_test[['protein_name']])
    protein_onehot_df = pd.DataFrame(protein_onehot, index=df_test.index)

    # Get MolFormer embeddings for combined SMILES in batches
    df_test['combined_smiles'] = df_test['molecule_smiles'] + ' ' + df_test['buildingblock1_smiles'] + ' ' + df_test['buildingblock2_smiles'] + ' ' + df_test['buildingblock3_smiles']
    combined_smiles_list = df_test['combined_smiles'].tolist()
    molformer_embeddings_list = []

    for i in range(0, len(combined_smiles_list), config['batch_size']):
        batch_smiles = combined_smiles_list[i:i + config['batch_size']]
        molformer_embeddings_list.extend(get_molformer_embeddings_batch(batch_smiles, tokenizer, model, device))

    df_test['molformer_embeddings'] = molformer_embeddings_list[:len(df_test)]

    # Combine ECFPs, MolFormer embeddings, and one-hot encoded protein names
    df_test['hybrid_embeddings'] = df_test.apply(
        lambda row: np.concatenate([row['ecfp'], row['molformer_embeddings'].flatten(), protein_onehot_df.loc[row.name]]), axis=1
    )

    # Final feature vectors
    hybrid_embeddings = np.vstack(df_test['hybrid_embeddings'].values).astype(np.float32)

    print(f"Test feature dimension: {hybrid_embeddings.shape[1]}")

    # Normalize embeddings
    faiss.normalize_L2(hybrid_embeddings)

    # Initialize FAISS index with GPU
    d = hybrid_embeddings.shape[1]  # Dimension of the embeddings
    res = faiss.StandardGpuResources() if config['use_gpu_for_faiss'] else None
    index = faiss.IndexFlatL2(d)  # Index for flat (exact) L2 distance
    if res:
        gpu_index = faiss.index_cpu_to_gpu(res, 0, index)  # Move index to GPU
    else:
        gpu_index = index

    # Add embeddings to the index
    gpu_index.add(hybrid_embeddings)

    # Retrieve the top k nearest neighbors for each embedding
    k = 5  # Number of nearest neighbors to retrieve
    D, I = gpu_index.search(hybrid_embeddings, k)

    # Apply attention mechanism and weighted averaging
    def apply_attention_and_weighted_averaging(embeddings, neighbors_idx, distances):
        augmented_embeddings = []
        for i, (neighbors, dist) in enumerate(zip(neighbors_idx, distances)):
            neighbor_embeddings = embeddings[neighbors]

            # Calculate attention weights
            similarities = 1 / (dist + 1e-6)
            attention_weights = torch.softmax(torch.tensor(similarities), dim=0).numpy()

            # Weighted averaging of neighbor embeddings
            weighted_average = np.average(neighbor_embeddings, axis=0, weights=attention_weights)

            # Combine the original embedding with the weighted average of neighbors
            final_embedding = np.concatenate([embeddings[i], weighted_average])
            augmented_embeddings.append(final_embedding)

        return np.array(augmented_embeddings)

    final_feature_vectors = apply_attention_and_weighted_averaging(hybrid_embeddings, I, D)

    # Ensure the test feature dimension matches the expected dimension
    if final_feature_vectors.shape[1] != expected_dim:
        raise ValueError(f"Test data dimension {final_feature_vectors.shape[1]} does not match training data dimension {expected_dim}")

    return final_feature_vectors

# Load the test data
test_file = '/kaggle/input/leash-BELKA/test.csv'
df_test_full = pd.read_csv(test_file)
df_test_sample = df_test_full.sample(frac=1, random_state=42).reset_index(drop=True)  # Adjust sample size for testing

# Prepare test data
hybrid_embeddings_test = prepare_test_data(df_test_sample, onehot_encoder, config, mf_tokenizer, mf_model, device, X_final.shape[1])

# Generate predictions using the first ensemble model
first_ensemble_predictions = ensemble_model.predict_proba(hybrid_embeddings_test)

# Since the test data does not have true labels, just show predictions
print("First Ensemble Model Predictions:")
print(first_ensemble_predictions)

# Save the predictions to a CSV file
output_file = 'submission.csv'
df_test_sample['predictions'] = first_ensemble_predictions[:, 1]
df_test_sample[['id', 'predictions']].to_csv(output_file, index=False)
print(f"Predictions saved to {output_file}")

# Function to generate submission
def generate_submission(test_file, output_file, ensemble_model, onehot_encoder, config, tokenizer, model, device):
    # Load the test data
    df_test_full = pd.read_csv(test_file)
    df_test_sample = df_test_full.sample(frac=1, random_state=42)
    df_test_full = generate_features(df_test_sample, onehot_encoder, config, tokenizer, model, device)
    
    hybrid_embeddings = np.vstack(df_test_full['hybrid_embeddings'].values).astype(np.float32)

    # Normalize embeddings
    faiss.normalize_L2(hybrid_embeddings)

    # Initialize FAISS index with GPU
    d = hybrid_embeddings.shape[1]  # Dimension of the embeddings
    res = faiss.StandardGpuResources() if config['use_gpu_for_faiss'] else None
    index = faiss.IndexFlatL2(d)  # Index for flat (exact) L2 distance
    if res:
        gpu_index = faiss.index_cpu_to_gpu(res, 0, index)  # Move index to GPU
    else:
        gpu_index = index

    # Add embeddings to the index
    gpu_index.add(hybrid_embeddings)

    # Retrieve the top k nearest neighbors for each embedding
    k = 5  # Number of nearest neighbors to retrieve
    D, I = gpu_index.search(hybrid_embeddings, k)

    # Apply attention mechanism and weighted averaging
    def apply_attention_and_weighted_averaging(embeddings, neighbors_idx, distances):
        augmented_embeddings = []
        for i, (neighbors, dist) in enumerate(zip(neighbors_idx, distances)):
            neighbor_embeddings = embeddings[neighbors]

            # Calculate attention weights
            similarities = 1 / (dist + 1e-6)
            attention_weights = torch.softmax(torch.tensor(similarities), dim=0).numpy()

            # Weighted averaging of neighbor embeddings
            weighted_average = np.average(neighbor_embeddings, axis=0, weights=attention_weights)

            # Combine the original embedding with the weighted average of neighbors
            final_embedding = np.concatenate([embeddings[i], weighted_average])
            augmented_embeddings.append(final_embedding)

        return np.array(augmented_embeddings)

    final_feature_vectors = apply_attention_and_weighted_averaging(hybrid_embeddings, I, D)

    # Generate predictions using the first ensemble model
    first_ensemble_predictions = ensemble_model.predict_proba(final_feature_vectors)[:, 1]

    # Run basic QC checks on the predictions
    if not pd.api.types.is_numeric_dtype(first_ensemble_predictions):
        raise ValueError('All submission values must be numeric')

    if not np.isfinite(first_ensemble_predictions).all():
        raise ValueError('All submission values must be finite')

    if first_ensemble_predictions.min() < 0:
        raise ValueError('All submission values must be at least zero')

    if first_ensemble_predictions.max() > 1:
        raise ValueError('All submission values must be no greater than one')

    # Create a DataFrame with 'id' and 'binds' columns
    output_df = pd.DataFrame({'id': df_test_full['id'], 'binds': first_ensemble_predictions})

    # Save predictions to CSV
    output_df.to_csv(output_file, index=False)
    print("Submission file generated successfully.")

# Generate submission
generate_submission(test_file, output_file, ensemble_model, onehot_encoder, config, mf_tokenizer, mf_model, device)
